In [1]:
#Basic libraries
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
# library used for over sampling the data
from imblearn.over_sampling import RandomOverSampler

In [2]:
data = pd.read_csv('E:/research/final_data_set.csv')

In [3]:
data.shape

(23482, 15)

In [4]:
data

,Age in 5-year groups,Husband/partner education level,Births in last three years,Contraceptive use and intention,Wanted last child,Did fieldworker talk about family planning,Visited by fieldworker in last 12 months,Respondent occupation,Getting medical help for self: getting money needed for treatment,Getting medical help for self: getting permission to go,Getting medical help for self: distance to health facility,media exposure,educational level,Wealth_index,terminated pregnancy
0,6,0,1,4,1.000000,0.000000,1,0,1.000000,2,1.000000,0,0,1,0
1,1,0,1,4,1.000000,1.000000,1,0,1.000000,2,2.000000,0,1,1,0
2,5,0,0,4,1.000000,0.000000,1,0,1.000000,2,1.000000,0,0,0,0
3,2,1,1,4,1.000000,0.000000,1,0,1.000000,1,1.000000,0,1,0,0
4,2,1,1,4,1.000000,0.000000,1,0,1.000000,1,1.000000,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23477,4,0,0,1,1.000000,0.000000,0,1,1.000000,1,1.000000,0,0,1,1
23478,5,1,0,1,1.000000,0.000000,0,1,1.156165,2,1.156165,1,0,2,1
23479,4,0,0,1,1.920441,0.920441,1,1,1.079559,2,2.000000,0,0,0,1
23480,5,3,0,3,1.000000,0.356367,1,0,1.356367,2,2.000000,3,1,2,1


In [4]:
data['terminated pregnancy'].value_counts()

terminated pregnancy
0    11741
1    11741
Name: count, dtype: int64

In [5]:
X=data.drop(['terminated pregnancy'],axis=1)
y=data['terminated pregnancy']

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [7]:
X_train.shape,y_train.shape, X_test.shape, y_test.shape

((18785, 14), (18785,), (4697, 14), (4697,))

# AdaBoostClassifier

In [11]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report,roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
base_estimator=DecisionTreeClassifier(max_depth=20,max_leaf_nodes=400,criterion='entropy', splitter='best', min_samples_split=2)
model = AdaBoostClassifier(base_estimator=base_estimator,n_estimators=400,learning_rate= 1.0)
model.fit(X_train, y_train)
pred = model.predict(X_test)
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.89      0.90      0.90      2347
           1       0.90      0.89      0.89      2350

    accuracy                           0.90      4697
   macro avg       0.90      0.90      0.90      4697
weighted avg       0.90      0.90      0.90      4697



In [12]:
accuracy= accuracy_score(y_test,pred)

In [13]:
accuracy

0.8950393868426655

In [14]:
confusion_matrix=confusion_matrix(y_test,pred)

In [15]:
confusion_matrix

array([[2121,  226],
       [ 267, 2083]], dtype=int64)

In [16]:
from sklearn.metrics import precision_score,recall_score
precision=precision_score(y_test, pred, pos_label='positive',average='macro')

In [17]:
precision

0.8951565427832101

In [18]:
recall=recall_score(y_test, pred,pos_label='positive',average='macro')

In [19]:
recall

0.8950449192722262

In [20]:
from sklearn import model_selection
kfold = model_selection.KFold(n_splits=10)
results = model_selection.cross_val_score(model, X,y.values.ravel(),scoring='accuracy', cv=kfold)
cross_val= results.mean()
cross_val

0.8874480097007462

In [21]:
cross_val= results.mean()

In [22]:
cross_val

0.8874480097007462

In [23]:
from sklearn.metrics import f1_score
f1_score=f1_score(y_test, pred,pos_label='positive',average='macro')

In [24]:
f1_score

0.8950325164610831

In [25]:
from sklearn.metrics import roc_auc_score

ROCovo = roc_auc_score(y, model.predict_proba(X)[:, 1])

In [26]:
ROCovo

0.9895252362946649

In [27]:
adaboosted_dt = pd.DataFrame({'accuracy':accuracy,
                   'precision':precision,'recall':recall,'cross_val':cross_val,'f1_score':f1_score,'ROCovo':ROCovo},
       columns=['accuracy','precision','recall','cross_val','f1_score','ROCovo'],index=["performance"])
adaboosted_dt

,accuracy,precision,recall,cross_val,f1_score,ROCovo
performance,0.895039,0.895157,0.895045,0.887448,0.895033,0.989525


# RandomForestClassifier

In [39]:
from sklearn.ensemble import RandomForestClassifier
kfold_validation= 5
RF = RandomForestClassifier(criterion= 'gini',
 max_depth= 20,
 max_features= 'sqrt',
 min_samples_split= 3,
 n_estimators= 400)
RF.fit(X_train, y_train)
pred = RF.predict(X_test)
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.89      0.92      0.91      2347
           1       0.92      0.89      0.90      2350

    accuracy                           0.91      4697
   macro avg       0.91      0.91      0.91      4697
weighted avg       0.91      0.91      0.91      4697



In [40]:
accuracy=accuracy_score(y_test,pred)
accuracy

0.9058973813072174

In [41]:
from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(y_test, pred)

In [42]:

confusion_matrix

array([[2159,  188],
       [ 254, 2096]], dtype=int64)

In [43]:
precision=precision_score(y_test, pred, pos_label='positive',average='macro')
precision

0.9062125541524564

In [44]:
recall=recall_score(y_test, pred,pos_label='positive',average='macro')
recall

0.9059063177075306

In [45]:
from sklearn import model_selection
kfold = model_selection.KFold(n_splits=10)
results = model_selection.cross_val_score(RF, X,y.values.ravel(),scoring='accuracy', cv=kfold)

In [46]:
cross_val=results.mean()
cross_val

0.8901297119438262

In [47]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, accuracy_score, roc_auc_score

f1_score=f1_score(y_test, pred,pos_label='positive',average='macro')
f1_score

0.9058804488562797

In [48]:
ROCovo=roc_auc_score(y, RF.predict_proba(X)[:, 1])
ROCovo

0.9914412676966966

In [49]:
adaboosted_RF = pd.DataFrame({'accuracy':accuracy,
                   'precision':precision,'recall':recall,'cross_val':cross_val,'f1_score':f1_score,'ROCovo':ROCovo},
       columns=['accuracy','precision','recall','cross_val','f1_score','ROCovo'],index=["performance"])
adaboosted_RF

,accuracy,precision,recall,cross_val,f1_score,ROCovo
performance,0.905897,0.906213,0.905906,0.89013,0.90588,0.991441


# XGBClassifier

In [40]:
from xgboost import XGBClassifier
xgb_clf = XGBClassifier(learning_rate=0.1, max_depth= 10, n_estimators= 300)
xgb_clf.fit(X_train, y_train)
pred = xgb_clf.predict(X_test)
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.89      0.91      0.90      2347
           1       0.91      0.89      0.90      2350

    accuracy                           0.90      4697
   macro avg       0.90      0.90      0.90      4697
weighted avg       0.90      0.90      0.90      4697



In [41]:
accuracy=accuracy_score(y_test,pred)
accuracy

0.8980200127741111

In [43]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, accuracy_score, roc_auc_score

print("",confusion_matrix(y_test,pred))

 [[2128  219]
 [ 260 2090]]


In [44]:
precision=precision_score(y_test, pred, pos_label='positive',average='macro')
precision

0.8981380121337161

In [45]:
recall=recall_score(y_test, pred,pos_label='positive',average='macro')
recall

0.8980255464196031

In [46]:
from sklearn import model_selection
kfold = model_selection.KFold(n_splits=10)
results = model_selection.cross_val_score(xgb_clf, X,y.values.ravel(),scoring='accuracy', cv=kfold)

In [47]:
cross_val=results.mean()
cross_val

0.8836998309476719

In [48]:
rocxgbovo=roc_auc_score(y, xgb_clf.predict_proba(X)[:,1])
rocxgbovo

0.9885622877342543

In [49]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, accuracy_score, roc_auc_score

f1_score=f1_score(y_test, pred,pos_label='positive',average='macro')
f1_score

0.8980133374946425

In [51]:
adaboosted_xgb = pd.DataFrame({'accuracy':accuracy,
                   'precision':precision,'recall':recall,'cross_val':cross_val,'f1_score':f1_score,'rocxgbovo':rocxgbovo},
       columns=['accuracy','precision','recall','cross_val','f1_score','rocxgbovo'],index=["performance"])
adaboosted_xgb

,accuracy,precision,recall,cross_val,f1_score,rocxgbovo
performance,0.89802,0.898138,0.898026,0.8837,0.898013,0.988562


# CatBoostClassifier

In [52]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(depth= 10, iterations= 400, learning_rate= 0.1)
model.fit(X_train,y_train,verbose=False, plot=True)
pred = model.predict(X_test)
print(classification_report(y_test,pred))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

              precision    recall  f1-score   support

           0       0.89      0.92      0.90      2347
           1       0.91      0.88      0.90      2350

    accuracy                           0.90      4697
   macro avg       0.90      0.90      0.90      4697
weighted avg       0.90      0.90      0.90      4697



In [53]:
accuracy=accuracy_score(y_test,pred)
accuracy

0.8992974238875878

In [54]:
print("",confusion_matrix(y_test,pred))

 [[2148  199]
 [ 274 2076]]


In [55]:
precision=precision_score(y_test, pred, pos_label='positive',average='macro')
precision

0.8996989138029601

In [56]:
recall=recall_score(y_test, pred,pos_label='positive',average='macro')
recall

0.8993075814303457

In [57]:
from sklearn import model_selection
kfold = model_selection.KFold(n_splits=10)
results = model_selection.cross_val_score(model, X,y.values.ravel(),scoring='accuracy', cv=kfold)

0:	learn: 0.6158416	total: 47.1ms	remaining: 18.8s
1:	learn: 0.5586454	total: 94.3ms	remaining: 18.8s
2:	learn: 0.5104511	total: 144ms	remaining: 19.1s
3:	learn: 0.4773990	total: 189ms	remaining: 18.8s
4:	learn: 0.4491176	total: 234ms	remaining: 18.5s
5:	learn: 0.4299200	total: 289ms	remaining: 19s
6:	learn: 0.4117918	total: 337ms	remaining: 18.9s
7:	learn: 0.3972043	total: 382ms	remaining: 18.7s
8:	learn: 0.3875691	total: 428ms	remaining: 18.6s
9:	learn: 0.3788582	total: 474ms	remaining: 18.5s
10:	learn: 0.3720807	total: 524ms	remaining: 18.5s
11:	learn: 0.3634776	total: 575ms	remaining: 18.6s
12:	learn: 0.3588268	total: 620ms	remaining: 18.5s
13:	learn: 0.3513621	total: 664ms	remaining: 18.3s
14:	learn: 0.3455030	total: 713ms	remaining: 18.3s
15:	learn: 0.3404336	total: 772ms	remaining: 18.5s
16:	learn: 0.3362792	total: 817ms	remaining: 18.4s
17:	learn: 0.3310402	total: 863ms	remaining: 18.3s
18:	learn: 0.3276471	total: 927ms	remaining: 18.6s
19:	learn: 0.3225327	total: 986ms	remaini

165:	learn: 0.1757243	total: 8.58s	remaining: 12.1s
166:	learn: 0.1751867	total: 8.63s	remaining: 12s
167:	learn: 0.1748474	total: 8.68s	remaining: 12s
168:	learn: 0.1743890	total: 8.72s	remaining: 11.9s
169:	learn: 0.1742255	total: 8.77s	remaining: 11.9s
170:	learn: 0.1740023	total: 8.81s	remaining: 11.8s
171:	learn: 0.1738575	total: 8.86s	remaining: 11.7s
172:	learn: 0.1732737	total: 8.91s	remaining: 11.7s
173:	learn: 0.1729964	total: 8.96s	remaining: 11.6s
174:	learn: 0.1722461	total: 9s	remaining: 11.6s
175:	learn: 0.1719313	total: 9.05s	remaining: 11.5s
176:	learn: 0.1717132	total: 9.1s	remaining: 11.5s
177:	learn: 0.1712390	total: 9.15s	remaining: 11.4s
178:	learn: 0.1706717	total: 9.2s	remaining: 11.4s
179:	learn: 0.1702295	total: 9.25s	remaining: 11.3s
180:	learn: 0.1695968	total: 9.3s	remaining: 11.3s
181:	learn: 0.1693651	total: 9.35s	remaining: 11.2s
182:	learn: 0.1691119	total: 9.4s	remaining: 11.1s
183:	learn: 0.1688270	total: 9.44s	remaining: 11.1s
184:	learn: 0.1684757	t

325:	learn: 0.1352526	total: 16.5s	remaining: 3.75s
326:	learn: 0.1351252	total: 16.6s	remaining: 3.7s
327:	learn: 0.1348410	total: 16.7s	remaining: 3.65s
328:	learn: 0.1347364	total: 16.7s	remaining: 3.6s
329:	learn: 0.1340913	total: 16.8s	remaining: 3.55s
330:	learn: 0.1339951	total: 16.8s	remaining: 3.5s
331:	learn: 0.1335008	total: 16.9s	remaining: 3.45s
332:	learn: 0.1332838	total: 16.9s	remaining: 3.4s
333:	learn: 0.1329058	total: 16.9s	remaining: 3.35s
334:	learn: 0.1327568	total: 17s	remaining: 3.3s
335:	learn: 0.1326306	total: 17s	remaining: 3.25s
336:	learn: 0.1324924	total: 17.1s	remaining: 3.19s
337:	learn: 0.1323465	total: 17.1s	remaining: 3.14s
338:	learn: 0.1321676	total: 17.2s	remaining: 3.09s
339:	learn: 0.1320364	total: 17.3s	remaining: 3.04s
340:	learn: 0.1319063	total: 17.3s	remaining: 2.99s
341:	learn: 0.1318026	total: 17.3s	remaining: 2.94s
342:	learn: 0.1317272	total: 17.4s	remaining: 2.89s
343:	learn: 0.1315518	total: 17.4s	remaining: 2.84s
344:	learn: 0.1314525

86:	learn: 0.2132606	total: 4.48s	remaining: 16.1s
87:	learn: 0.2125005	total: 4.54s	remaining: 16.1s
88:	learn: 0.2112658	total: 4.6s	remaining: 16.1s
89:	learn: 0.2112132	total: 4.61s	remaining: 15.9s
90:	learn: 0.2111903	total: 4.62s	remaining: 15.7s
91:	learn: 0.2105275	total: 4.67s	remaining: 15.6s
92:	learn: 0.2091726	total: 4.72s	remaining: 15.6s
93:	learn: 0.2087468	total: 4.76s	remaining: 15.5s
94:	learn: 0.2078211	total: 4.81s	remaining: 15.5s
95:	learn: 0.2073912	total: 4.86s	remaining: 15.4s
96:	learn: 0.2066919	total: 4.91s	remaining: 15.3s
97:	learn: 0.2057420	total: 4.96s	remaining: 15.3s
98:	learn: 0.2051952	total: 5.01s	remaining: 15.2s
99:	learn: 0.2046892	total: 5.06s	remaining: 15.2s
100:	learn: 0.2038812	total: 5.11s	remaining: 15.1s
101:	learn: 0.2031307	total: 5.16s	remaining: 15.1s
102:	learn: 0.2020235	total: 5.22s	remaining: 15s
103:	learn: 0.2016931	total: 5.26s	remaining: 15s
104:	learn: 0.2006708	total: 5.32s	remaining: 14.9s
105:	learn: 0.2000445	total: 5.

246:	learn: 0.1436887	total: 12.9s	remaining: 8.02s
247:	learn: 0.1435325	total: 13s	remaining: 7.97s
248:	learn: 0.1432093	total: 13.1s	remaining: 7.92s
249:	learn: 0.1430617	total: 13.1s	remaining: 7.88s
250:	learn: 0.1428826	total: 13.2s	remaining: 7.84s
251:	learn: 0.1426024	total: 13.3s	remaining: 7.79s
252:	learn: 0.1421655	total: 13.3s	remaining: 7.74s
253:	learn: 0.1419398	total: 13.4s	remaining: 7.7s
254:	learn: 0.1416362	total: 13.4s	remaining: 7.65s
255:	learn: 0.1411342	total: 13.5s	remaining: 7.6s
256:	learn: 0.1406501	total: 13.6s	remaining: 7.54s
257:	learn: 0.1404922	total: 13.6s	remaining: 7.48s
258:	learn: 0.1403074	total: 13.6s	remaining: 7.43s
259:	learn: 0.1402127	total: 13.7s	remaining: 7.38s
260:	learn: 0.1400929	total: 13.7s	remaining: 7.32s
261:	learn: 0.1399299	total: 13.8s	remaining: 7.26s
262:	learn: 0.1398339	total: 13.8s	remaining: 7.21s
263:	learn: 0.1397099	total: 13.9s	remaining: 7.15s
264:	learn: 0.1395800	total: 13.9s	remaining: 7.09s
265:	learn: 0.13

...

22:	learn: 0.3271923	total: 3.83s	remaining: 1m 2s
23:	learn: 0.3235796	total: 4.01s	remaining: 1m 2s
24:	learn: 0.3191872	total: 4.19s	remaining: 1m 2s
25:	learn: 0.3168962	total: 4.41s	remaining: 1m 3s
26:	learn: 0.3148459	total: 4.55s	remaining: 1m 2s
27:	learn: 0.3116174	total: 4.78s	remaining: 1m 3s
28:	learn: 0.3090280	total: 4.93s	remaining: 1m 3s
29:	learn: 0.3071921	total: 5.08s	remaining: 1m 2s
30:	learn: 0.3041733	total: 5.27s	remaining: 1m 2s
31:	learn: 0.3024006	total: 5.44s	remaining: 1m 2s
32:	learn: 0.3006319	total: 5.59s	remaining: 1m 2s
33:	learn: 0.2980795	total: 5.75s	remaining: 1m 1s
34:	learn: 0.2959955	total: 5.91s	remaining: 1m 1s
35:	learn: 0.2931680	total: 6.11s	remaining: 1m 1s
36:	learn: 0.2907431	total: 6.31s	remaining: 1m 1s
37:	learn: 0.2886693	total: 6.49s	remaining: 1m 1s
38:	learn: 0.2868798	total: 6.64s	remaining: 1m 1s
39:	learn: 0.2849631	total: 6.79s	remaining: 1m 1s
40:	learn: 0.2843156	total: 6.94s	remaining: 1m
41:	learn: 0.2825221	total: 7.1s	r

184:	learn: 0.1796709	total: 30.1s	remaining: 35s
185:	learn: 0.1795626	total: 30.3s	remaining: 34.8s
186:	learn: 0.1790472	total: 30.5s	remaining: 34.7s
187:	learn: 0.1786369	total: 30.7s	remaining: 34.6s
188:	learn: 0.1782789	total: 30.8s	remaining: 34.4s
189:	learn: 0.1781738	total: 30.9s	remaining: 34.2s
190:	learn: 0.1779245	total: 31.1s	remaining: 34s
191:	learn: 0.1775186	total: 31.3s	remaining: 33.9s
192:	learn: 0.1771276	total: 31.4s	remaining: 33.7s
193:	learn: 0.1766131	total: 31.5s	remaining: 33.5s
194:	learn: 0.1760215	total: 31.7s	remaining: 33.4s
195:	learn: 0.1758689	total: 31.9s	remaining: 33.2s
196:	learn: 0.1754968	total: 32.1s	remaining: 33s
197:	learn: 0.1752109	total: 32.2s	remaining: 32.8s
198:	learn: 0.1749211	total: 32.4s	remaining: 32.7s
199:	learn: 0.1747120	total: 32.5s	remaining: 32.5s
200:	learn: 0.1745669	total: 32.7s	remaining: 32.4s
201:	learn: 0.1739722	total: 32.9s	remaining: 32.2s
202:	learn: 0.1737636	total: 33.1s	remaining: 32.1s
203:	learn: 0.1734

343:	learn: 0.1393643	total: 57.1s	remaining: 9.3s
344:	learn: 0.1391913	total: 57.3s	remaining: 9.13s
345:	learn: 0.1390190	total: 57.5s	remaining: 8.97s
346:	learn: 0.1388680	total: 57.6s	remaining: 8.8s
347:	learn: 0.1384614	total: 57.7s	remaining: 8.63s
348:	learn: 0.1380089	total: 57.9s	remaining: 8.46s
349:	learn: 0.1377330	total: 58.1s	remaining: 8.3s
350:	learn: 0.1373652	total: 58.3s	remaining: 8.14s
351:	learn: 0.1368994	total: 58.5s	remaining: 7.98s
352:	learn: 0.1366823	total: 58.6s	remaining: 7.81s
353:	learn: 0.1364915	total: 58.8s	remaining: 7.64s
354:	learn: 0.1359967	total: 58.9s	remaining: 7.47s
355:	learn: 0.1356643	total: 59.1s	remaining: 7.3s
356:	learn: 0.1353508	total: 59.3s	remaining: 7.14s
357:	learn: 0.1352016	total: 59.4s	remaining: 6.97s
358:	learn: 0.1350933	total: 59.6s	remaining: 6.81s
359:	learn: 0.1347483	total: 59.8s	remaining: 6.64s
360:	learn: 0.1343963	total: 60s	remaining: 6.48s
361:	learn: 0.1341900	total: 1m	remaining: 6.32s
362:	learn: 0.1339801

105:	learn: 0.2139879	total: 17.6s	remaining: 48.7s
106:	learn: 0.2136401	total: 17.7s	remaining: 48.5s
107:	learn: 0.2131385	total: 17.9s	remaining: 48.4s
108:	learn: 0.2127802	total: 18.1s	remaining: 48.4s
109:	learn: 0.2120773	total: 18.3s	remaining: 48.2s
110:	learn: 0.2114338	total: 18.5s	remaining: 48.1s
111:	learn: 0.2110442	total: 18.6s	remaining: 47.9s
112:	learn: 0.2103313	total: 18.8s	remaining: 47.9s
113:	learn: 0.2095905	total: 19s	remaining: 47.7s
114:	learn: 0.2093169	total: 19.1s	remaining: 47.4s
115:	learn: 0.2086426	total: 19.3s	remaining: 47.3s
116:	learn: 0.2082160	total: 19.5s	remaining: 47.1s
117:	learn: 0.2077110	total: 19.6s	remaining: 46.9s
118:	learn: 0.2072717	total: 19.8s	remaining: 46.7s
119:	learn: 0.2068086	total: 20s	remaining: 46.7s
120:	learn: 0.2065154	total: 20.2s	remaining: 46.6s
121:	learn: 0.2061110	total: 20.4s	remaining: 46.4s
122:	learn: 0.2041338	total: 20.5s	remaining: 46.2s
123:	learn: 0.2031378	total: 20.7s	remaining: 46s
124:	learn: 0.2027

264:	learn: 0.1533130	total: 46.2s	remaining: 23.5s
265:	learn: 0.1532145	total: 46.4s	remaining: 23.4s
266:	learn: 0.1528990	total: 46.6s	remaining: 23.2s
267:	learn: 0.1526957	total: 46.7s	remaining: 23s
268:	learn: 0.1522099	total: 46.9s	remaining: 22.9s
269:	learn: 0.1520621	total: 47.2s	remaining: 22.7s
270:	learn: 0.1517565	total: 47.4s	remaining: 22.6s
271:	learn: 0.1516508	total: 47.6s	remaining: 22.4s
272:	learn: 0.1515586	total: 47.8s	remaining: 22.3s
273:	learn: 0.1514036	total: 48.1s	remaining: 22.1s
274:	learn: 0.1508096	total: 48.4s	remaining: 22s
275:	learn: 0.1503611	total: 48.6s	remaining: 21.9s
276:	learn: 0.1500283	total: 48.9s	remaining: 21.7s
277:	learn: 0.1497462	total: 49.1s	remaining: 21.5s
278:	learn: 0.1495777	total: 49.3s	remaining: 21.4s
279:	learn: 0.1493091	total: 49.5s	remaining: 21.2s
280:	learn: 0.1491813	total: 49.6s	remaining: 21s
281:	learn: 0.1489172	total: 49.8s	remaining: 20.8s
282:	learn: 0.1487339	total: 50s	remaining: 20.7s
283:	learn: 0.148514

In [58]:
corss_val=results.mean()
corss_val

0.8849773327734518

In [59]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, accuracy_score, roc_auc_score

f1_score=f1_score(y_test, pred,pos_label='positive',average='macro')
f1_score

0.8992737556176431

In [60]:
ROCcatovo=roc_auc_score(y, model.predict_proba(X)[:,1])
ROCcatovo

0.9884066886642695

In [61]:
cat = pd.DataFrame({'accuracy':accuracy,
                   'precision':precision,'recall':recall,'cross_val':cross_val,'f1_score':f1_score,'ROCcatovo':ROCcatovo},
       columns=['accuracy','precision','recall','cross_val','f1_score','ROCcatovo'],index=["performance"])
cat

,accuracy,precision,recall,cross_val,f1_score,ROCcatovo
performance,0.899297,0.899699,0.899308,0.8837,0.899274,0.988407


# ExtraTreesClassifier

In [29]:

import pandas
from sklearn import model_selection
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score


model52 = ExtraTreesClassifier(n_estimators=200, max_features=10, random_state=0, n_jobs=-1)
model52= model52.fit(X_train, y_train) 
model52.score(X_test, y_test)
                                  
y_pred = model52.predict(X_test)

kfold_validation= 5
results=cross_val_score(model52,X_train, y_train,cv=kfold_validation)
print(results)
print(np.mean(results))



print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))                              


[0.87117381 0.88288528 0.87010913 0.89087038 0.88528081]
0.8800638807559222
[[2103  244]
 [ 271 2079]]
              precision    recall  f1-score   support

           0       0.89      0.90      0.89      2347
           1       0.89      0.88      0.89      2350

    accuracy                           0.89      4697
   macro avg       0.89      0.89      0.89      4697
weighted avg       0.89      0.89      0.89      4697



In [30]:
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.890355546093251

In [31]:
print("",confusion_matrix(y_test,y_pred))

 [[2103  244]
 [ 271 2079]]


In [32]:
precision=precision_score(y_test, y_pred, pos_label='positive',average='macro')
precision

0.8904050408337416

In [33]:
recall=recall_score(y_test, y_pred,pos_label='positive',average='macro')
recall

0.8903591728689408

In [34]:
from sklearn import model_selection
kfold = model_selection.KFold(n_splits=10)
results = model_selection.cross_val_score(model, X,y.values.ravel(),scoring='accuracy', cv=kfold)

In [35]:
corss_val=results.mean()
corss_val

0.8881716856569508

In [36]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, accuracy_score, roc_auc_score

f1_score=f1_score(y_test, y_pred,pos_label='positive',average='macro')
f1_score

0.890352683369964

In [37]:
ROCcatovo=roc_auc_score(y, model.predict_proba(X)[:,1])
ROCcatovo

0.9895252362946649

In [38]:
extra = pd.DataFrame({'accuracy':accuracy,
                   'precision':precision,'recall':recall,'cross_val':cross_val,'f1_score':f1_score,'ROCcatovo':ROCcatovo},
       columns=['accuracy','precision','recall','cross_val','f1_score','ROCcatovo'],index=["performance"])
extra

,accuracy,precision,recall,cross_val,f1_score,ROCcatovo
performance,0.890356,0.890405,0.890359,0.887448,0.890353,0.989525


# DecisionTreeClassifier

In [51]:
from sklearn.tree import DecisionTreeClassifier
Dt = DecisionTreeClassifier(criterion= 'gini',
 max_depth= 30,
 max_features= 'auto',
 max_leaf_nodes= 500,
 min_samples_split= 3)
Dt.fit(X_train, y_train)
pred = Dt.predict(X_test)
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.78      0.85      0.81      2347
           1       0.84      0.76      0.80      2350

    accuracy                           0.81      4697
   macro avg       0.81      0.81      0.81      4697
weighted avg       0.81      0.81      0.81      4697



In [52]:
accuracy=accuracy_score(y_test,pred)
accuracy

0.8060464126037896

In [53]:
 print("",confusion_matrix(y_test,pred))

 [[2001  346]
 [ 565 1785]]


In [55]:
precision=precision_score(y_test, pred, pos_label='positive',average='macro')
precision

0.8087239258059313

In [56]:
recall=recall_score(y_test, pred,pos_label='positive',average='macro')
recall

0.80607611346309

In [57]:
from sklearn import model_selection
kfold = model_selection.KFold(n_splits=10)
results = model_selection.cross_val_score(model, X,y.values.ravel(),scoring='accuracy', cv=kfold)
corss_val=results.mean()
corss_val

0.8879590829545794

In [58]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, accuracy_score, roc_auc_score

f1_score=f1_score(y_test, pred,pos_label='positive',average='macro')
f1_score

0.8056353730541868

In [59]:
ROCcatovo=roc_auc_score(y, model.predict_proba(X)[:,1])
ROCcatovo

0.9895252362946649

In [60]:
extra = pd.DataFrame({'accuracy':accuracy,
                   'precision':precision,'recall':recall,'cross_val':cross_val,'f1_score':f1_score,'ROCcatovo':ROCcatovo},
       columns=['accuracy','precision','recall','cross_val','f1_score','ROCcatovo'],index=["performance"])
extra

,accuracy,precision,recall,cross_val,f1_score,ROCcatovo
performance,0.806046,0.808724,0.806076,0.89013,0.805635,0.989525


In [ ]:
from lightgbm import LGBMClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split



# Create an LGBM classifier
classifier = LGBMClassifier(num_leaves=31, learning_rate=0.1, n_estimators=100)

# Fit the classifier to the training data
classifier.fit(X_train, y_train)

# Extract feature importances
feature_importances = classifier.feature_importances_

# Print feature importances
for feature, importance in zip(range(len(feature_importances)), feature_importances):
    print(f"Feature {feature}: Importance {importance}")